# A3C for Kung Fu

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


### Importing the libraries

In [2]:
import cv2
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.distributions as distributions
from torch.distributions import Categorical
import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.spaces import Box

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [3]:
# implement the network class
class Network(nn.Module):

  def __init__(self, action_size):
    # activate super inheritance
    super(Network, self).__init__()
    # define convolutional layers
    self.conv1 = torch.nn.Conv2d(in_channels = 4, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv2 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)
    self.conv3 = torch.nn.Conv2d(in_channels = 32, out_channels = 32, kernel_size = (3,3), stride = 2)

    # create the flattening layer
    self.flatten = torch.nn.Flatten()

    # first full connected layer
    self.fc1 = torch.nn.Linear(512, 128)

    # output layers (action values and state values)
    self.fc2a = torch.nn.Linear(128, action_size)
    self.fc2s = torch.nn.Linear(128, 1)

# define the forward method (Forward Propagation Signal)
  def forward(self, state):
    x = self.conv1(state)
    # rectifier activation function (ReLU)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    # fwd prop. for third conv. layer
    x = self.conv3(x)
    x = F.relu(x)

    # flattening layer fwd. prop.
    x = self.flatten(x)
    x = self.fc1(x)
    x = F.relu(x)

    # output layers
    action_values = self.fc2a(x)
    state_values = self.fc2s(x)[0]
    return action_values, state_values

## Part 2 - Training the AI

### Setting up the environment

In [4]:
class PreprocessAtari(ObservationWrapper):

  def __init__(self, env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4):
    super(PreprocessAtari, self).__init__(env)
    self.img_size = (height, width)
    self.crop = crop
    self.dim_order = dim_order
    self.color = color
    self.frame_stack = n_frames
    n_channels = 3 * n_frames if color else n_frames
    obs_shape = {'tensorflow': (height, width, n_channels), 'pytorch': (n_channels, height, width)}[dim_order]
    self.observation_space = Box(0.0, 1.0, obs_shape)
    self.frames = np.zeros(obs_shape, dtype = np.float32)

  def reset(self):
    self.frames = np.zeros_like(self.frames)
    obs, info = self.env.reset()
    self.update_buffer(obs)
    return self.frames, info

  def observation(self, img):
    img = self.crop(img)
    img = cv2.resize(img, self.img_size)
    if not self.color:
      if len(img.shape) == 3 and img.shape[2] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = img.astype('float32') / 255.
    if self.color:
      self.frames = np.roll(self.frames, shift = -3, axis = 0)
    else:
      self.frames = np.roll(self.frames, shift = -1, axis = 0)
    if self.color:
      self.frames[-3:] = img
    else:
      self.frames[-1] = img
    return self.frames

  def update_buffer(self, obs):
    self.frames = self.observation(obs)

def make_env():
  env = gym.make("KungFuMasterDeterministic-v0", render_mode = 'rgb_array')
  env = PreprocessAtari(env, height = 42, width = 42, crop = lambda img: img, dim_order = 'pytorch', color = False, n_frames = 4)
  return env

env = make_env()

state_shape = env.observation_space.shape
number_actions = env.action_space.n
print("Observation shape:", state_shape)
print("Number actions:", number_actions)
print("Action names:", env.env.env.get_action_meanings())

/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


Observation shape: (4, 42, 42)
Number actions: 14
Action names: ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'DOWNRIGHT', 'DOWNLEFT', 'RIGHTFIRE', 'LEFTFIRE', 'DOWNFIRE', 'UPRIGHTFIRE', 'UPLEFTFIRE', 'DOWNRIGHTFIRE', 'DOWNLEFTFIRE']


/usr/local/lib/python3.10/dist-packages/gymnasium/core.py:311: UserWarning: WARN: env.get_action_meanings to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.get_action_meanings` for environment variables or `env.get_wrapper_attr('get_action_meanings')` that will search the reminding wrappers.
  logger.warn(


### Initializing the hyperparameters

In [5]:
# three hyperparameters to initialize
learning_rate = 1e-4
discount_factor = 0.99
number_of_environments = 10

### Implementing the A3C class

In [6]:
# implementing the agent class w/ softmax policy
class Agent():

  def __init__(self, action_size):
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.action_size = action_size
    # implementing the "brain"
    self.network = Network(action_size).to(self.device)
    # the optimizer that updates the weights / models of the brain
    self.optimizer = torch.optim.Adam(self.network.parameters(), lr = learning_rate)

  # implement the act method with a softmax strategy
  def act(self, state):
    # state must be in a batch
    if state.ndim == 3:
      state = [state] # ensures the state is in a batch
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    # call network on state to get action values and calling the fwd method
    action_values, _ = self.network(state)
    # using softmax policy to select a certain action
    policy = F.softmax(action_values, dim=-1)
    return np.array([np.random.choice(len(p), p = p) for p in policy.detach().cpu().numpy()])

  # implementing the step method
  def step(self, state, action, reward, next_state, done): # these args are batches
    # get batch size
    batch_size = state.shape[0]
    # converting the batches into torch tensors
    state = torch.tensor(state, dtype = torch.float32, device = self.device)
    next_state = torch.tensor(next_state, dtype = torch.float32, device = self.device)
    reward = torch.tensor(reward, dtype = torch.float32, device = self.device)
    done = torch.tensor(done, dtype = torch.bool, device = self.device).to(dtype = torch.float32)
    # action values and state values
    action_values, state_values = self.network(state)
    _, next_state_value = self.network(next_state)
    # compute target state value using bellman equation
    target_state_value = reward + discount_factor * next_state_value * (1 - done)
    # implementing A3C model
    advantage = target_state_value - state_values
    # computing total loss that will be back propagated (actor loss and critic loss)
    # we need the entropy and dist. of probability over the action values
    probs = F.softmax(action_values, dim=-1)
    logprobs = F.log_softmax(action_values, dim=-1)
    # now we can compute the entropy
    entropy = -torch.sum(probs * logprobs, axis = -1)
    # log probability of the actions that are actually selected using an array w/ batch size
    batch_idx = np.arange(batch_size)
    logp_actions = logprobs[batch_idx, action]
    # computing the actor loss + encourage exploration
    actor_loss = -(logp_actions * advantage.detach()).mean() - 0.001 * entropy.mean()
    # critic loss
    critic_loss = F.mse_loss(target_state_value.detach(), state_values)
    # total loss
    total_loss = actor_loss + critic_loss
    # back prop. into the NN and update weights, but reset before
    self.optimizer.zero_grad()
    total_loss.backward()
    # update model parameters aka weights w/ the optimizer
    self.optimizer.step()

### Initializing the A3C agent

In [7]:
agent = Agent(number_actions)

### Evaluating our A3C agent on a single episode

In [8]:
def evaluate(agent, env, n_episodes = 1):
  # will return a list of the total rewards in each episode
  episodes_rewards = []
  for _ in range(n_episodes):
    # initialize the state w/ reset method
    state, _ = env.reset()
    total_reward = 0
    while True:
      # agent must play an action
      action = agent.act(state)
      state, reward, done, info, _ = env.step(action[0])
      # compute the accumulated reward
      total_reward += reward
      if done:
        break
    # append the total reward to the episode list
    episodes_rewards.append(total_reward)

  # return the list of accumulated rewards of the episodes
  return episodes_rewards

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Testing multiple agents on multiple environments at the same time

In [9]:
class EnvBatch:

  def __init__(self, n_envs = 10):
    self.envs = [make_env() for _ in range(n_envs)]

  # reset method
  def reset(self):
    # create a new list of initialized states
    _states = []
    # populate the list
    for env in self.envs:
      _states.append(env.reset()[0])
    # return the list in the form of a NumPy array
    return np.array(_states)

  # step in multiple environments
  def step(self, actions):
    # for multiiple agents to step in multiple envs, we make a for loop w/ step func.
    # group all elements into NumPy arrays
    next_states, rewards, dones, infos, _ = map(np.array, zip(*[env.step(a) for env, a in zip(self.envs, actions)]))
    # check if any env have the done = True
    for i in range(len(self.envs)):
      if dones[i]:
        next_states[i] = self.envs[i].reset()[0]
    return next_states, rewards, dones, infos


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Training the A3C agent

In [10]:
import tqdm # progress bar for loops

# create instance of EnvBatch class
env_batch = EnvBatch(number_of_environments)
# reset batch of states in each environment
batch_states = env_batch.reset()

# implementing the progress bar
with tqdm.trange(0, 3001) as progress_bar:
  for i in progress_bar:
    # agents must play an action all at the same time in each env
    batch_actions = agent.act(batch_states)
    # next state and get reward, know whether episode is done or not
    batch_next_states, batch_rewards, batch_dones, _ = env_batch.step(batch_actions)
    # calling the step method from agent class
    batch_rewards *= 0.01 # stabilize the training
    agent.step(batch_states, batch_actions, batch_rewards, batch_next_states, batch_dones)
    # update batch of states to be the batch of next states
    batch_states = batch_next_states
    # print at every 1000 iterations
    if i % 1000 == 0:
      print("Average agent reward: ", np.mean(evaluate(agent, env, n_episodes=10)))


/usr/local/lib/python3.10/dist-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment KungFuMasterDeterministic-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
  0%|          | 0/3001 [00:00<?, ?it/s]<ipython-input-6-975c1a1247ff>:52: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  critic_loss = F.mse_loss(target_state_value.detach(), state_values)
<ipython-input-6-975c1a1247ff>:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  state = torch.tensor(state, dtype = torch.float32, device = self.device)
  0%|          | 5/3001 [00:40<5:00:16,  6.01

Average agent reward:  510.0


 33%|███▎      | 1005/3001 [01:49<1:20:19,  2.41s/it]

Average agent reward:  760.0


 67%|██████▋   | 2005/3001 [03:02<38:20,  2.31s/it]

Average agent reward:  780.0


100%|██████████| 3001/3001 [04:09<00:00, 12.03it/s]

Average agent reward:  560.0


## Part 3 - Visualizing the results

In [15]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(agent, env):
  state, _ = env.reset()
  done = False
  frames = []
  while not done:
    frame = env.render()
    frames.append(frame)
    action = agent.act(state)
    state, reward, done, _, _ = env.step(action[0])
  env.close()
  imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, env)

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()